In [7]:
from concurrent.futures import ThreadPoolExecutor
from aws.AWS import AWSClient, bucket, queue_url
from time import sleep
import psutil
import os
import glob
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
def check_all_threads_complete(futures):
    for i,future in enumerate(futures):
        if future.done():
            futures.pop(i)
            ctr += 1
    return (ctr ==len(futures))

In [3]:
def run_darknet(video_path, output_path):
    darknet_dir = "../darknet/"
    os.chdir(darknet_dir)
    if os.path.exists(output_path):
        os.remove(output_path)
    cmd = "./darknet detector demo cfg/coco.data cfg/yolov3-tiny.cfg yolov3-tiny.weights {} >> {}"
    os.system(cmd.format(video_path, output_path))

NameError: name 'aws' is not defined

In [19]:
pool = ThreadPoolExecutor(3)
futures = []
aws = AWSClient(auth=True)
instance_status = aws.get_python_object_s3(bucket, 'status')
InstID = 'i-017231cbb330871ed'
#if InstID not in instance_status.keys():
instance_status[InstID] = -1

flag = 0
aws.put_python_object_s3(bucket, 'status', instance_status)

True

In [ ]:
while(True):
    #do we need to limit the number of threads as well
    proc_usage = (psutil.cpu_percent() +  psutil.cpu_percent() +  psutil.cpu_percent())/3.0
    if(proc_usage >= 80 ):
        #set flag=1 for instance = InstID in S3
        flag = 1
        #add_message_to_queue(current_instance_id,instance_queue_url)
    if aws.get_queue_length(queue_url) == 0:
        if(check_all_threads_complete(futures)):
            #set flag=-1 for instance = InstID in S3
            flag=-1
            instance_status[InstID] = -1
            aws.put_python_object_s3(bucket, 'status', instance_status)
            aws.switch_off_ec2_instance(InstID)
            #ec2.instances.filter(InstanceIds=InstID).stop()
    else:
        video_name = aws.get_message_sqs_download_video_from_s3(bucket, queue_url)
        video_name = os.getcwd() + '/' + video_name
        ouput_file = os.getcwd() + '/' + video_name.split('/')[1].split('.')[0] + '.txt'
        future = pool.submit(run_darknet, video_name, output_file)
        futures.append(future)
        #send_video_to_sqs_s3('sample.h264',s3_bucket_name,queue_url)
        #for i in range(1,5):
        print('submitted')

In [49]:
aws.get_python_object_s3(bucket, 'output/2020-03-27_20-31-13.h264')

'bird'

In [60]:
aws.get_python_object_s3(bucket, 'status')

{'i-0f7ef001ba37a53fc': -1,
 'i-004f91d178feb2dd9': 1,
 'i-038007690f166efc8': -1,
 'i-03a0cc5a88c554d48': -1,
 'i-03f222747002d442d': -1,
 'i-09c6ccb85c393bb36': -1,
 'i-0f5327a4feffe15d8': -1,
 'i-0c53a48e5ac876ecf': -1,
 'i-017231cbb330871ed': -1}

In [16]:
a = {
 'i-017231cbb330871ed': -1,
 'i-06597d7d31aaf9899': -1}
aws.put_python_object_s3(bucket, 'status', dict(a))

True

In [59]:
aws.get_python_object_s3(bucket,'output/2020-03-27_21-02-13.h264')

'bird'

In [55]:
aws.get_python_object_s3(bucket,'status')

{'i-03f222747002d442d': -1,
 'i-0c53a48e5ac876ecf': -1,
 'i-017231cbb330871ed': 1,
 'i-0f7ef001ba37a53fc': -1,
 'i-038007690f166efc8': -1,
 'i-09c6ccb85c393bb36': 1,
 'i-0f5327a4feffe15d8': 1,
 'i-03a0cc5a88c554d48': 1,
 'i-004f91d178feb2dd9': -1}

In [53]:
d = {'i-017231cbb330871ed': -1,
 'i-038007690f166efc8': -1,
 'i-03a0cc5a88c554d48': -1,
 'i-09c6ccb85c393bb36': -1,
 'i-0f5327a4feffe15d8': -1,
 'i-0c53a48e5ac876ecf': -1,
 'i-0f7ef001ba37a53fc': -1,
 'i-004f91d178feb2dd9': -1,
 'i-03f222747002d442d': -1}
aws.put_python_object_s3(bucket, 'status', d)


True

In [7]:
os.chdir('videos')
for file in glob.glob('*.h264'):
    print (file)
    aws.upload_video_s3_send_message_sqs(file, bucket, queue_url)
os.chdir('../')

2020-03-19_00-38-54.h264
2020-03-19_00-38-38.h264
2020-03-19_00-39-43.h264
2020-03-19_00-39-43__1.h264
2020-03-19_00-38-38__1.h264
2020-03-19_00-38-59.h264
2020-03-19_00-38-59__1.h264
2020-03-19_00-38-54__1.h264


In [37]:
aws.get_queue_length(queue_url)

7

In [23]:
os.chdir('../')

In [3]:
aws.update_instance_status('i-06597d7d31aaf9899', -1)

{'i-0912a5fdc78485f46': -1,
 'i-017231cbb330871ed': -1,
 'i-06597d7d31aaf9899': -1}

In [30]:
aws.switch_on_ec2_instance('i-017231cbb330871ed')

{'StartingInstances': [{'CurrentState': {'Code': 0, 'Name': 'pending'},
   'InstanceId': 'i-017231cbb330871ed',
   'PreviousState': {'Code': 80, 'Name': 'stopped'}}],
 'ResponseMetadata': {'RequestId': '43e637e7-b00f-4c14-bf31-1e65569ab5c9',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '43e637e7-b00f-4c14-bf31-1e65569ab5c9',
   'content-type': 'text/xml;charset=UTF-8',
   'content-length': '579',
   'date': 'Sun, 29 Mar 2020 14:04:58 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}

In [31]:
aws.switch_off_ec2_instance('i-017231cbb330871ed')

{'StoppingInstances': [{'CurrentState': {'Code': 64, 'Name': 'stopping'},
   'InstanceId': 'i-017231cbb330871ed',
   'PreviousState': {'Code': 16, 'Name': 'running'}}],
 'ResponseMetadata': {'RequestId': '36a87d4c-8fb0-4046-b55e-f73195bb4772',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '36a87d4c-8fb0-4046-b55e-f73195bb4772',
   'content-type': 'text/xml;charset=UTF-8',
   'content-length': '579',
   'date': 'Sun, 29 Mar 2020 14:16:29 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}

In [9]:
out = aws.ec2_client.describe_instances()

In [21]:
def reset_instances_status(self):
    instance_map = {}
    controller_id = 'i-0912a5fdc78485f46'
    for inst in out['Reservations']:
        for ins in inst['Instances']:
            if(ins['InstanceId'])!=controller_id:
                instance_map[ins['InstanceId']]=-1
    self.update_instance_status(instance_map)

In [51]:
aws.reset_instances_status()